In [1]:
import numpy as np
import pandas as pd

In [2]:
datasetpath = '/home/pedro/datasets/jbrj_herbarium/occurrence.txt'

In [561]:
colnames = ['recordedBy', 'taxonRank', 'scientificName', 'eventDate',
            'occurrenceID', 'collectionCode', 'stateProvince', 
            'locality', 'countryCode', 'county']
occs = pd.read_csv(datasetpath, sep='\t', usecols=colnames)

Here I'm interested in records from the Itatiaia municipality with information about the collectors.

In [562]:
occs_itatiaia = occs[occs['county'].apply(lambda x: 'Itatiaia' in str(x))]
occs_itatiaia = occs_itatiaia[occs_itatiaia['recordedBy'].notnull()]

occs_itatiaia.head(10)

,collectionCode,occurrenceID,recordedBy,eventDate,countryCode,stateProvince,county,locality,scientificName,taxonRank
169,RB,urn:catalog:JBRJ:RB:327442,R.W.Kaempfe,1931-03-03,BR,Rio de Janeiro,Itatiaia,Serra da Mantiqueira.,ROSACEAE Fragaria,genus
225,RB,urn:catalog:JBRJ:RB:379779,P. Campos Porto,1918-08-20,BR,Rio de Janeiro,Itatiaia,Lactiferos.,SAPOTACEAE,family
306,ITA,urn:catalog:JBRJ:RB:477401,José Adaltro Lemes,1977-10-04,BR,Rio de Janeiro,Itatiaia,Macieiras.,SYMPLOCACEAE,family
310,ITA,urn:catalog:JBRJ:RB:478924,J. J. Sampaio,1942-09-25,BR,Rio de Janeiro,Itatiaia,Repouso Itatiaiai.,SOLANACEAE Solanum,genus
408,RUSU,urn:catalog:JBRJ:RB:679739,"Camerik, A.M.",1975-04-23,BR,Rio de Janeiro,Itatiaia,NaN,AQUIFOLIACEAE Ilex,genus
514,RB,urn:catalog:JBRJ:RB:554980,M. Salazar Yepes,NaN,BR,Rio de Janeiro,Itatiaia,"PARNA-Itatiaia, trilha três Picos.",INDETERMINADA,family
517,RB,urn:catalog:JBRJ:RB:560431,M. Salazar Yepes & A.A. Carvalho Jr.,NaN,BR,Rio de Janeiro,Itatiaia,"PARNA-Itatiaia, na estrada Portaria Principal-...",INDETERMINADA,family
705,RB,urn:catalog:JBRJ:RB:1006065,M. Salazar Yepes & A.A. Carvalho Jr.,2007-8-7,BR,Rio de Janeiro,Itatiaia,NaN,PUCCINIALES Phakopsora rossmanii,specie
741,RB,urn:catalog:JBRJ:RB:145779,A. Mello,NaN,BR,Rio de Janeiro,Itatiaia,NaN,LEGUMINOSAE Melanoxylon brauna Schott,specie
1253,ITA,urn:catalog:JBRJ:RB:477461,P. C. Porto,1918,BR,Rio de Janeiro,Itatiaia,NaN,SAPINDACEAE,family


In [642]:
# functions defined on notebook 5

import re
import unicodedata, string

def namesFromString( namesStr, splitOn=';', unique=False, preserveOrder=False ):
    if type(splitOn)==str:
        namesSplit = namesStr.split(splitOn)
        
    elif type(splitOn)==list:
        namesSplit = re.split( '|'.join( c for c in splitOn ) ,namesStr)
    
    namesList = [ n for n in [ name.strip() for name in namesSplit ] if n!='' ]
    
    if unique:
        if not preserveOrder: return list(set(namesList))
        else:
            namesCounts = dict( (n,0) for n in namesList )
            unique_namesList = []
            for n in namesList:
                if namesCounts[n]: continue
                namesCounts[n]+=1
                unique_namesList.append(n)
        return unique_namesList   
    
    return namesList


def normalize(name, normalizationForm='NFKD'):
    name = name.lower() # to lowecase
    name = name.replace('.','') # remove periods
    name_ls = tuple( part.strip() for part in name.split(',') ) # split and strip names into tuples

    normalize = lambda s: ''.join( x for x in unicodedata.normalize(normalizationForm, s) if x in string.ascii_letters ) # remove accents
    name_ls = tuple( normalize(name) for name in name_ls )
    
    return ','.join(name_ls)


def get_names_indices( df, names_col, names_map ):
    # FIXME This function does not allow other delimiters on names string
    names = set( n for names in df[names_col].apply(lambda x: namesFromString(str(x))) for n in names )
    df = df.copy()
    df[names_col] = df[names_col].astype(str).apply( namesFromString )
    
    names_indices = dict( (name,[]) for name in names_map.values() )
    for i,names in df[names_col].iteritems():
        for name in names:
            if names_map is not None:
                names_indices[names_map[name]].append(i)
            else:
                names_indices[name].append(i)
            
    return names_indices

def getNamesIndexes( df, split_names_col, names_map ):
    # split_names_col is a column with names already split
    namesIndexes = dict( (name,[]) for name in names_map.values() )
    for i,names in df[split_names_col].iteritems():
        for name in names:
            if names_map is not None:
                namesIndexes[names_map[name]].append(i)
            else:
                namesIndexes[name].append(i)
            
    return namesIndexes

Let's first create a `collectors_split` column, which stores the atomized collectors names as extracted by an `atomizeNames` expression.

In [563]:
atomizeNames = lambda ns: namesFromString(ns, splitOn=['&',';',' e ']) 
occs_itatiaia['collectors_split'] = occs_itatiaia['recordedBy'].apply( atomizeNames )

Here I'll work only with **collaborative records**, which were obtained by more than one collector

In [347]:
occs_itatiaia_collaborative = occs_itatiaia[occs_itatiaia['collectors_split'].apply(lambda l: len(l)>1)]
occs_itatiaia_collaborative.head()

,collectionCode,occurrenceID,recordedBy,eventDate,countryCode,stateProvince,county,locality,scientificName,taxonRank,collectors_split
517,RB,urn:catalog:JBRJ:RB:560431,M. Salazar Yepes & A.A. Carvalho Jr.,NaN,BR,Rio de Janeiro,Itatiaia,"PARNA-Itatiaia, na estrada Portaria Principal-...",INDETERMINADA,family,"[M. Salazar Yepes, A.A. Carvalho Jr.]"
705,RB,urn:catalog:JBRJ:RB:1006065,M. Salazar Yepes & A.A. Carvalho Jr.,2007-8-7,BR,Rio de Janeiro,Itatiaia,NaN,PUCCINIALES Phakopsora rossmanii,specie,"[M. Salazar Yepes, A.A. Carvalho Jr.]"
2044,RB,urn:catalog:JBRJ:RB:1005724,M. Salazar Yepes; A.A. Carvalho Jr. & C.M. Sak...,2006-12-3,BR,Rio de Janeiro,Itatiaia,NaN,PUCCINIALES Skierka cristata Mains,specie,"[M. Salazar Yepes, A.A. Carvalho Jr., C.M. Sak..."
2083,RB,urn:catalog:JBRJ:RB:1004480,M. Salazar Yepes & A.A. Carvalho Jr.,2007-8-6,BR,Rio de Janeiro,Itatiaia,NaN,PUCCINIALES Puccinia eupatorii Dietel,specie,"[M. Salazar Yepes, A.A. Carvalho Jr.]"
2096,RB,urn:catalog:JBRJ:RB:1005872,M. Salazar Yepes & A.A. Carvalho Jr.,2007-9-10,BR,Rio de Janeiro,Itatiaia,NaN,PUCCINIALES Phakopsora meibomiae (Arthur) Arthur,specie,"[M. Salazar Yepes, A.A. Carvalho Jr.]"


The percentage of collaborative records on the dataset from Itatiaia is relatively low:

In [302]:
"Percentage of collaborative records at Itatiaia: {:.2%}".format(
    occs_itatiaia_collaborative.shape[0]/occs_itatiaia.shape[0])

'Percentage of collaborative records at Itatiaia: 8.03%'

---

Let's get a list of names of collaborative collectors in Itatiaia. I'll only show the first ten elements:

In [564]:
namesList = sorted(list(set( n for nlist in occs_itatiaia_collaborative['collectors_split'] for n in nlist )))
namesList[:10]

['A. C. Brade',
 'A.A. Carvalho Jr.',
 'A.A.Carvalho',
 'A.C.Mota',
 'Aparicio',
 'Apparicio',
 'Brade',
 'Brade, A. C.',
 'C.M. Sakuragui',
 'D. P. Costa']

From these names I can then build a names map, which maps each variation of a name to its normal form. I'll show the first ten elements mapped.

In [565]:
namesMap = dict( (n, normalize(n)) for n in namesList )
[ "{} -- maps to --> {}".format(s,t) for (s,t) in namesMap.items()][:10]

['A. C. Brade -- maps to --> acbrade',
 'A.A. Carvalho Jr. -- maps to --> aacarvalhojr',
 'A.A.Carvalho -- maps to --> aacarvalho',
 'A.C.Mota -- maps to --> acmota',
 'Aparicio -- maps to --> aparicio',
 'Apparicio -- maps to --> apparicio',
 'Brade -- maps to --> brade',
 'Brade, A. C. -- maps to --> brade,ac',
 'C.M. Sakuragui -- maps to --> cmsakuragui',
 'D. P. Costa -- maps to --> dpcosta']

Now, we can inspect the names map and verify that some variations actually belong to the same entity. I'll use the function below to remap normalized names in the names map. The function takes a remapping dictionary and the names map to be modified and performs the remapping inplace.

In [350]:
def remap(remappingDict, nmap):
    # remaps names after normalized (using a remapping dictionary)
    getNamePrimitives = lambda n: ( name for name,norm in nmap.items() if norm == n )
    for n,t in ( (n,t) for s,t in remappingDict.items() for n in getNamePrimitives(s) ):
        nmap[n]=t

In [351]:
remappingDict = {
    'aacarvalhojr': 'aacarvalho',
    'aparicio': 'apparicio',
    'brade': 'brade,ac',
    'fidalgo,k': 'fidalgo',
    'fildalgo': 'fidalgo',
    'fidalgo': 'oswaldofidalgo',
    'kauffmannfidalfoeg': 'kauffmannfidalgo',
    'mekauffmannfidalgo': 'kauffmannfidalgo',
    'kfidalgo': 'kauffmannfidalgo',

    'lanstsyak,l': 'lanstyak,l',
    'fidalgo,o': 'oswaldofidalgo',
    'ofidalgo': 'oswaldofidalgo',
    'silveira,o': 'silveiramelo,o',
    'tamandare': 'tamandaredetoledo',
    'toledo': 'tamandaredetoledo',

    'pjmmaas': 'pjmaas',
    'pjlmaas': 'pjmaas',
    'acbrade': 'brade,ac',
    
}

remap(remappingDict, namesMap)
sorted(list(set(namesMap.values())))
[ "{} -- maps to --> {}".format(s,t) for (s,t) in namesMap.items()][:10]

['A. C. Brade -- maps to --> brade,ac',
 'A.A. Carvalho Jr. -- maps to --> aacarvalho',
 'A.A.Carvalho -- maps to --> aacarvalho',
 'A.C.Mota -- maps to --> acmota',
 'Aparicio -- maps to --> apparicio',
 'Apparicio -- maps to --> apparicio',
 'Brade -- maps to --> brade,ac',
 'Brade, A. C. -- maps to --> brade,ac',
 'C.M. Sakuragui -- maps to --> cmsakuragui',
 'D. P. Costa -- maps to --> dpcosta']

Now let's get a **names index** which stores the ids of the rows where each normalized name appears:

In [566]:
names_index = get_names_indices(occs_itatiaia_collaborative, 'collectors_split', namesMap)

But it turns out that some of the names are still not normalized, as it is the case of `'msalazaryepes,aacarvalhojr'`:

In [316]:
list(namesMap.values())[30:35]

['mbarros',
 'msalazaryepes',
 'msalazaryepes,aacarvalhojr',
 'mdmviannafilho',
 'kauffmannfidalgo']

In that case an alternative is to renormalize some records and get another names from the updated normalized names column:

In [357]:
def reatomizeRecords(df, columnName, recordsIndex, atomizingOperation):
    df.loc[recordsIndex,columnName] = df.loc[recordsIndex,columnName].apply(operation)

In [356]:
operation = lambda l: [ n for name in l for n in namesFromString(name,',')]
reatomizeRecords(occs_itatiaia, 'collectors_split', names_index['msalazaryepes,aacarvalhojr'], operation)

['M. Salazar Yepes', 'A.A. Carvalho Jr.', 'F. Santoro']

---

---

# A little bit of cleaning first...

## Atomizing names

In [721]:
occs_itatiaia = occs[occs['county'].apply(lambda x: 'Itatiaia' in str(x))]
occs_itatiaia = occs_itatiaia[occs_itatiaia['recordedBy'].notnull()]

In [722]:
def atomizeNames( df, colName, atomizingFunction, recordsIndex=None, atomizedColName=None  ):
    if not atomizedColName:
        atomizedColName = colName+"_atomized"
    
    if recordsIndex:
        df.loc[recordsIndex,atomizedColName] = df.loc[recordsIndex,colName].apply( atomizingFunction )
    else: # atomize the entire dataframe
        df.loc[:,atomizedColName] = df.loc[:,colName].apply( atomizingFunction )

In [723]:
atomizingOperation = lambda ns: namesFromString(ns, splitOn=['&',';',' e ']) 

atomizeNames(occs_itatiaia, 'recordedBy', atomizingOperation)
occs_itatiaia.head()

,collectionCode,occurrenceID,recordedBy,eventDate,countryCode,stateProvince,county,locality,scientificName,taxonRank,recordedBy_atomized
169,RB,urn:catalog:JBRJ:RB:327442,R.W.Kaempfe,1931-03-03,BR,Rio de Janeiro,Itatiaia,Serra da Mantiqueira.,ROSACEAE Fragaria,genus,[R.W.Kaempfe]
225,RB,urn:catalog:JBRJ:RB:379779,P. Campos Porto,1918-08-20,BR,Rio de Janeiro,Itatiaia,Lactiferos.,SAPOTACEAE,family,[P. Campos Porto]
306,ITA,urn:catalog:JBRJ:RB:477401,José Adaltro Lemes,1977-10-04,BR,Rio de Janeiro,Itatiaia,Macieiras.,SYMPLOCACEAE,family,[José Adaltro Lemes]
310,ITA,urn:catalog:JBRJ:RB:478924,J. J. Sampaio,1942-09-25,BR,Rio de Janeiro,Itatiaia,Repouso Itatiaiai.,SOLANACEAE Solanum,genus,[J. J. Sampaio]
408,RUSU,urn:catalog:JBRJ:RB:679739,"Camerik, A.M.",1975-04-23,BR,Rio de Janeiro,Itatiaia,NaN,AQUIFOLIACEAE Ilex,genus,"[Camerik, A.M.]"


## The NamesMap class

In [724]:
from copy import deepcopy

class NamesMap:
    _map=None
    _normalizationFunction=None
    _remappingDict=None
    
    def __init__(self, names, normalizationFunction):
        self._normalizationFunction = normalizationFunction
        self._map = dict( (n, self._normalizationFunction(n)) for n in names )
        return
    
    def clearMap(self):
        self._map={}
    
    def remap(self, remappingDict):
        if self._remappingDict is None:
            self._remappingDict = remappingDict
        else:
            self._remappingDict.update(remappingDict)
        return     
    
    def insertNames(self, names, normalizationFunction=None, rebuild=False):
        # if rebuild is true, the entire map (but not the remapping dict) is rebuilt from scratch
        if rebuild==True:
            self.clearMap()
        if normalizationFunction is None: 
            normalizationFunction = self._normalizationFunction
        self._map.update( dict( (n,normalizationFunction(n)) for n in names ) )
        return
    
    def removeNames(self, names):
        # removes names from the map 
        pass
    
    def getMap(self, remap=True):
        # Returns a COPY of the map
        # If remap is set to true, some remapping occurrs
        res = deepcopy(self._map)
        if remap and self._remappingDict is not None:
            getNamesPrimitives = lambda n: ( name for name,norm in self._map.items() if norm == n )
            for n,t in ( (n,t) for s,t in self._remappingDict.items() for n in getNamesPrimitives(s) ):
                res[n]=t
        return res
    
    def getNormalizedNames(self, remap=True):
        return sorted(list(set(self.getMap(remap=remap).values())))
    
    def getNamePrimitives(self, n):
        nmap = self.getMap()
        return [ name for name,norm in nmap.items() if norm == n ]
        

---

In [725]:
occs_itatiaia_collaborative = occs_itatiaia[occs_itatiaia['recordedBy_atomized'].apply(lambda l: len(l)>1)]
namesList = sorted(list(set( n for nlst in occs_itatiaia_collaborative['recordedBy_atomized'] for n in nlst )))

# create names map
nm = NamesMap(namesList, normalize)
remappingDict = {
    'aacarvalhojr': 'aacarvalho',
    'aparicio': 'apparicio',
    'brade': 'brade,ac',
    'fidalgo,k': 'kauffmannfidalgo',
    'fildalgo': 'oswaldofidalgo',
    'fidalgo': 'oswaldofidalgo',
    'kauffmannfidalfoeg': 'kauffmannfidalgo',
    'mekauffmannfidalgo': 'kauffmannfidalgo',
    'kfidalgo': 'kauffmannfidalgo',
    'lanstsyak,l': 'lanstyak,l',
    'fidalgo,o': 'oswaldofidalgo',
    'ofidalgo': 'oswaldofidalgo',
    'silveira,o': 'silveiramelo,o',
    'tamandare': 'tamandaredetoledo',
    'toledo': 'tamandaredetoledo',
    'pjmmaas': 'pjmaas',
    'pjlmaas': 'pjmaas',
    'acbrade': 'brade,ac',
}
nm.remap(remappingDict)
nm.remap({'profpilger':'pilger'})

In [726]:
nm.getMap()

{'A. C. Brade': 'brade,ac',
 'A.A. Carvalho Jr.': 'aacarvalho',
 'A.A.Carvalho': 'aacarvalho',
 'A.C.Mota': 'acmota',
 'Aparicio': 'apparicio',
 'Apparicio': 'apparicio',
 'Brade': 'brade,ac',
 'Brade, A. C.': 'brade,ac',
 'C.M. Sakuragui': 'cmsakuragui',
 'D. P. Costa': 'dpcosta',
 'D.P.Costa': 'dpcosta',
 'Ed. Pereira, Egle,': 'edpereira,egle,',
 'Edmundo': 'edmundo',
 'F. Santoro': 'fsantoro',
 'F. Toledo Jr.': 'ftoledojr',
 'Fidalgo, K.': 'kauffmannfidalgo',
 'Fidalgo, O.': 'oswaldofidalgo',
 'Fildalgo': 'oswaldofidalgo',
 'G. Martinelli': 'gmartinelli',
 'Graziela': 'graziela',
 'I. França': 'ifranca',
 'K.Fidalgo': 'kauffmannfidalgo',
 'Kauffmann Fidalfo Eg-07': 'kauffmannfidalgo',
 'Kauffmann Fidalgo': 'kauffmannfidalgo',
 'L. C. S. Giordano': 'lcsgiordano',
 'L. S. Sylvestre': 'lssylvestre',
 'L.A.F': 'laf',
 'Lanstsyak, L.': 'lanstyak,l',
 'Lanstyak, L.': 'lanstyak,l',
 'Luiz': 'luiz',
 'M. Barros': 'mbarros',
 'M. Salazar Yepes': 'msalazaryepes',
 'M. Salazar Yepes, A.A. Carv

---

## Reatomizing records

### Names index

In [727]:
def getNamesIndexes( df, atomizedNamesCol, namesMap=None ):
    # split_names_col is a column with names already split
    namesIndexes = dict( (name,[]) for name in namesMap.values() )
    for i,names in df[atomizedNamesCol].iteritems():
        for name in names:
            if namesMap is not None:
                try:
                    namesIndexes[namesMap[name]].append(i)
                except KeyError:
                    pass
            else:
                namesIndexes[name].append(i)
            
    return namesIndexes

In [728]:
ni = getNamesIndexes(occs_itatiaia, 'recordedBy_atomized', nm.getMap())

These records are the ones to be atomized

In [729]:
names_to_atomize = ['msalazaryepes,aacarvalhojr', 'edpereira,egle,' ]

In [730]:
atomizingOperation = lambda l: [ n for name in l for n in namesFromString(name,',')]

for n in names_to_atomize:
    atomizeNames(occs_itatiaia,'recordedBy_atomized',atomizingOperation,ni[n], 'recordedBy_atomized')

Now we rebuild the names map, while keeping the remapping dictionary

In [731]:
occs_itatiaia_collaborative = occs_itatiaia[occs_itatiaia['recordedBy_atomized'].apply(lambda l: len(l)>1)]
namesList = sorted(list(set( n for nlst in occs_itatiaia_collaborative['recordedBy_atomized'] for n in nlst )))

In [732]:
nm.insertNames(namesList,rebuild=True)
ni = getNamesIndexes(occs_itatiaia, 'recordedBy_atomized', nm.getMap())

---

## Joining names

Some records should not have been atomized:

In [733]:
occs_itatiaia_collaborative.loc[ni['laf']]

,collectionCode,occurrenceID,recordedBy,eventDate,countryCode,stateProvince,county,locality,scientificName,taxonRank,recordedBy_atomized
64066,RB,urn:catalog:JBRJ:RB:573945,L.A.F; Santos Filho,2009-09-21,BR,Rio de Janeiro,Itatiaia,Em fragmento florestal no Alto dos Brejos. Par...,MELASTOMATACEAE Tibouchina,genus,"[L.A.F, Santos Filho]"


In [734]:
def mergeNames( df, colName, joiningFunction, recordsIndex ):
    df.loc[recordsIndex,colName] = df.loc[recordsIndex,colName].apply(joiningFunction)

In [735]:
joinOp = lambda l: [','.join(l)]
mergeNames( occs_itatiaia, 'recordedBy_atomized',joinOp, ni['laf'] )

In [744]:
occs_itatiaia_collaborative = occs_itatiaia[occs_itatiaia['recordedBy_atomized'].apply(lambda l: len(l)>1)]
namesList = sorted(list(set( n for nlst in occs_itatiaia_collaborative['recordedBy_atomized'] for n in nlst )))
nm.insertNames(namesList,rebuild=True)
ni = getNamesIndexes(occs_itatiaia, 'recordedBy_atomized', nm.getMap())

In [747]:
nm.getNormalizedNames()

['aacarvalho',
 'acmota',
 'apparicio',
 'brade,ac',
 'cmsakuragui',
 'dpcosta',
 'edmundo',
 'edpereira',
 'egle',
 'fsantoro',
 'ftoledojr',
 'gmartinelli',
 'graziela',
 'ifranca',
 'kauffmannfidalgo',
 'lanstyak,l',
 'lcsgiordano',
 'lssylvestre',
 'luiz',
 'mbarros',
 'mdmviannafilho',
 'mpmdelima',
 'msalazaryepes',
 'occhioni,p',
 'oswaldofidalgo',
 'pilger',
 'pjmaas',
 'silveiramelo,o',
 'srgradstein',
 'ssalgado',
 'tamandaredetoledo']

In [760]:
occs_itatiaia.loc[ni['brade,ac']]

,collectionCode,occurrenceID,recordedBy,eventDate,countryCode,stateProvince,county,locality,scientificName,taxonRank,recordedBy_atomized
54635,RB,urn:catalog:JBRJ:RB:210617,Brade,NaN,BR,Rio de Janeiro,Itatiaia,Pedra da Divisa.,MALPIGHIACEAE Banisteriopsis scutellata (Grise...,specie,[Brade]
54852,RB,urn:catalog:JBRJ:RB:230984,Brade,1937-3,BR,Rio de Janeiro,Itatiaia,NaN,MELASTOMATACEAE Tibouchina cordifolia Cogn.,specie,[Brade]
54883,RB,urn:catalog:JBRJ:RB:231944,"Brade, A. C.",1945-2-18,BR,Rio de Janeiro,Itatiaia,Serra Taquaral.,MELASTOMATACEAE Miconia doriana Cogn.,specie,"[Brade, A. C.]"
54905,RB,urn:catalog:JBRJ:RB:232791,"Brade, A. C.",1934-9,BR,Rio de Janeiro,Itatiaia,Monte Serrat.,MELASTOMATACEAE Miconia urophylla DC.,specie,"[Brade, A. C.]"
54908,RB,urn:catalog:JBRJ:RB:232824,"Brade, A. C.",1948-2-12,BR,Rio de Janeiro,Itatiaia,Lote 17.,MELASTOMATACEAE Leandra amplexicaulis DC.,specie,"[Brade, A. C.]"
54909,RB,urn:catalog:JBRJ:RB:232829,"Brade, A. C.",1948-2-5,BR,Rio de Janeiro,Itatiaia,Lote 28-30.,MELASTOMATACEAE Leandra mosenii Cogn.,specie,"[Brade, A. C.]"
54910,RB,urn:catalog:JBRJ:RB:232843,Brade,1948-1-3,BR,Rio de Janeiro,Itatiaia,"Estrada do Rio Itatiaia, Lago Azul. Itatiaia",MELASTOMATACEAE Leandra hirta Raddi,specie,[Brade]
54913,RB,urn:catalog:JBRJ:RB:232873,"Brade, A. C.",1945-2-20,BR,Rio de Janeiro,Itatiaia,"Maromba, nova picada.",MELASTOMATACEAE Leandra melastomoides Raddi,specie,"[Brade, A. C.]"
54916,RB,urn:catalog:JBRJ:RB:232901,Brade,1945-2-23,BR,Rio de Janeiro,Itatiaia,Lote do Almirante.,MELASTOMATACEAE Leandra fragilis Cogn.,specie,[Brade]
54917,RB,urn:catalog:JBRJ:RB:232912,Brade,1934-9,BR,Rio de Janeiro,Itatiaia,Macieiras.,MELASTOMATACEAE Leandra sublanata Cogn.,specie,[Brade]
